<a href="https://colab.research.google.com/github/simonbustamante/mit-algoritmos-de-regresion/blob/master/Caso_de_Estudio_2_3_Crecen_m%C3%A1s_r%C3%A1pido_los_pa%C3%ADses_pobres_que_los_ricos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Caso de Estudio 2.3 - ¿Crecen más rápido los países pobres que los ricos?


Sincronice su cuenta de Google. Para ello, siga el link que aparece en la salida de la siguiente celda una vez ejecutada. Copie el código que le aparece en pantalla e introdúzcalo en la salida de la celda. Una vez vea el mensaje: `Google Drive sincronizado con éxito!` puede continuar ejecutando el resto de celdas.

In [1]:
from google.colab import auth
auth.authenticate_user()

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
data_drop = drive.CreateFile({'id':'1wAZ5JZuq2_zKqzDQlIuOxUSTfk9N_tq5'})
data_drop.GetContentFile('growth.Rdata')

print('Google Drive sincronizado con éxito!')

Google Drive sincronizado con éxito!


In [2]:
import rpy2
import rpy2.robjects

In [3]:
%load_ext rpy2.ipython

Instalar e importar librerías: (ignorar resultados a no ser que no se muestre la frase: `¡Librerías instaladas con éxito!`)

In [4]:
%%R
install.packages("hdm")

cat('\n¡Librerías instaladas con éxito!')

(as ‘lib’ is unspecified)

















































	‘/tmp/RtmpqXmI0y/downloaded_packages’




¡Librerías instaladas con éxito!

In [5]:
%%R
# Importar librerías
library(hdm)

cat('\n¡Librerías importadas con éxito!')


¡Librerías importadas con éxito!

## Datos


In [6]:
%%R
# Cargar la base de datos
load(file='growth.Rdata')

# Ver las variables de la base de datos
class(growth)
str(growth)

# Mostrar dimensiones de la base de datos
dims  <- dim(growth)
cat('\nDimensiones de la base de datos:',toString(dims),'\n',fill = TRUE)

'data.frame':	90 obs. of  63 variables:
 $ Outcome  : num  -0.0243 0.1005 0.0671 0.0641 0.0279 ...
 $ intercept: int  1 1 1 1 1 1 1 1 1 1 ...
 $ gdpsh465 : num  6.59 6.83 8.9 7.57 7.16 ...
 $ bmp1l    : num  0.284 0.614 0 0.2 0.174 ...
 $ freeop   : num  0.153 0.314 0.204 0.249 0.299 ...
 $ freetar  : num  0.04389 0.06183 0.00919 0.03627 0.03737 ...
 $ h65      : num  0.007 0.019 0.26 0.061 0.017 0.023 0.039 0.024 0.402 0.145 ...
 $ hm65     : num  0.013 0.032 0.325 0.07 0.027 0.038 0.063 0.035 0.488 0.173 ...
 $ hf65     : num  0.001 0.007 0.201 0.051 0.007 0.006 0.014 0.013 0.314 0.114 ...
 $ p65      : num  0.29 0.91 1 1 0.82 0.5 0.92 0.69 1 1 ...
 $ pm65     : num  0.37 1 1 1 0.85 0.55 0.94 0.69 1 1 ...
 $ pf65     : num  0.21 0.65 1 1 0.81 0.5 0.92 0.69 1 1 ...
 $ s65      : num  0.04 0.16 0.56 0.24 0.17 0.08 0.17 0.14 0.9 0.28 ...
 $ sm65     : num  0.06 0.23 0.62 0.22 0.15 0.1 0.21 0.14 0.9 0.26 ...
 $ sf65     : num  0.02 0.09 0.51 0.31 0.13 0.07 0.12 0.13 0.9 0.4 ...
 $ fert65

In [7]:
%%R
write.csv(growth,'prueba.csv')


In [8]:
%%R
# Obtener los nombres de las variables
varnames= colnames(growth)
# Extraer los nombres de los controles y del regresor objetivo de varnames
xnames     <- varnames[-c(1,2,3)]     # nombres de las variables de X
dandxnames <- varnames[-c(1,2)]       # nombres de las variables de X y D

## Metodología

In [9]:
%%R
# crear fórmulas "pegando" nombres (así se guardan los nombre a mostrar)
fmla      <-  as.formula(paste("Outcome ~ ", paste(dandxnames, collapse= "+")))
full.fit  <-  lm(fmla, data=growth)
fmla.y    <-  as.formula(paste("Outcome ~ ", paste(xnames, collapse= "+")))
fmla.d    <-  as.formula(paste("gdpsh465~ ", paste(xnames, collapse= "+")))


# d parcial e y parcial por regresión lineal
rY       <- rlasso(fmla.y, data =growth)$res
rD       <- rlasso(fmla.d, data =growth)$res


# regresión lineal de Y parcialmente extraida en la D parcialmente extraída
partial.fit.lasso <- lm(rY~rD-1)

# crear tabla para recoger los resultados
table      <- matrix(0, 2, 4)
table[1,1:2]  <- summary(full.fit)$coef["gdpsh465",1:2]
table[2,1:2]  <- summary(partial.fit.lasso)$coef[1,1:2]
table[1,3:4]  <- confint(full.fit, level=0.95, 'gdpsh465')[1:2]
table[2,3:4]  <- confint(partial.fit.lasso, level=0.95, 'rD')[1:2]

# asignar nombre de columnas y filas
colnames(table) <- c("Estimación (beta)", "Error estándar","2.5%","97.5%")
rownames(table) <- c("Mínimos cuadrados ord. (OLS)", "Extracción parcial con Lasso")

## Resultados

In [10]:
%%R
# Mostrar resultados
print(table, digits=2)

                             Estimación (beta) Error estándar   2.5%  97.5%
Mínimos cuadrados ord. (OLS)           -0.0094          0.030 -0.071  0.052
Extracción parcial con Lasso           -0.0498          0.014 -0.077 -0.022
